In [1]:
import datetime as dt
import logging
import sys

import polars as pl

from backtest_lib.backtest import Backtest
from backtest_lib.examples.get_sp500_historical import get_sp500_market_view
from backtest_lib.market import MarketView
from backtest_lib.market.polars_impl import PolarsPastView, SeriesUniverseMapping
from backtest_lib.portfolio import WeightedPortfolio
from backtest_lib.strategy import Decision

logging.basicConfig(
    level=logging.INFO,
    stream=sys.stdout,
    format="[%(levelname)s] %(name)s: %(message)s",
    force=True,
)

market_view = get_sp500_market_view(
    start=dt.datetime.fromisoformat("2020-01-03"),
)
# market_view = pkl.load(open("2010-sp500-market-view.pkl", "rb"))
# market_view = market_view.truncated_to(500)


universe = tuple(market_view.prices.close.securities)

[*********************100%***********************]  617 of 617 completed

[ERROR] yfinance: 
55 Failed downloads:
[ERROR] yfinance: ['DISCK', 'XLNX', 'RTN', 'NBL', 'JWN', 'ATVI', 'XEC', 'DISCA', 'CXO', 'SIVB', 'KSU', 'ANSS', 'CERN', 'TWTR', 'CTXS', 'JNPR', 'COG', 'FLIR', 'CTLT', 'FBHS', 'ARNC', 'DFS', 'WLTW', 'PBCT', 'RE', 'GPS', 'JEC', 'FLT', 'DLPH', 'MRO', 'ADS', 'HES', 'ETFC', 'DISH', 'CDAY', 'PXD', 'CCR', 'NLSN', 'MXIM', 'HRS', 'DRE', 'TIF', 'VAR', 'ABMD', 'ALXN', 'FRC', 'WCG', 'FSR', 'HFC']: YFTzMissingError('possibly delisted; no timezone found')
[ERROR] yfinance: ['WBA']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-03 00:00:00 -> 2025-11-23 19:49:00.160637) (Yahoo error = "No data found, symbol may be delisted")')
[ERROR] yfinance: ['LUK', 'KORS', 'TSO', 'JOYG', 'WLP']: YFPricesMissingError('possibly delisted; no price data found  (1d 2020-01-03 00:00:00 -> 2025-11-23 19:49:00.160637)')


In [2]:
pf = WeightedPortfolio(
    cash=0,
    holdings=SeriesUniverseMapping.from_names_and_data(
        universe, pl.Series([1 / len(universe)] * len(universe))
    ),
)


def get_even_portfolio(full_universe, partial_universe: list[str]) -> WeightedPortfolio:
    zeroed_securities = list(set(full_universe) - set(partial_universe))
    return WeightedPortfolio(
        cash=0,
        holdings=SeriesUniverseMapping.from_names_and_data(
            partial_universe + zeroed_securities,
            pl.Series(
                ([1 / len(partial_universe)] * len(partial_universe))
                + ([0.0] * len(zeroed_securities))
            ),
        ),
    )


def index_strategy(universe, current_portfolio, market, ctx) -> Decision:
    tradable = [x for x, y in market.tradable.by_period[-1].items() if y]
    even_portfolio = get_even_portfolio(universe, tradable)

    return Decision(even_portfolio)


df = market_view.prices.close.by_security.as_df()
cols = df.columns
numeric_cols = [col for col in cols if df.get_column(col).dtype.is_numeric()]
ewma = (
    df.with_columns(
        pl.col(col).ewm_mean(com=2).alias(f"{col}_EWM") for col in numeric_cols
    )
    .with_columns((pl.col(col) / pl.col(f"{col}_EWM")) for col in numeric_cols)
    .select(cols)
)

ewma_past_view = PolarsPastView.from_dataframe(ewma)

market_view = MarketView(
    prices=market_view.prices,
    tradable=market_view.tradable,
    signals={"ewma_ratio": ewma_past_view},
)


def ewma_strategy(universe, current_portfolio, market, ctx):
    tradable_today = market.tradable.by_period[-1]
    latest_ewma_ratio = (
        market.signals["ewma_ratio"]
        .by_security[[sec for sec in universe if tradable_today[sec] is True]]
        .by_period[-1]
    )

    avg_ewma_ratio = latest_ewma_ratio.mean()
    norm_ewma_ratio = latest_ewma_ratio / avg_ewma_ratio
    weights = norm_ewma_ratio / len(norm_ewma_ratio)
    return Decision(target=WeightedPortfolio(cash=0, holdings=weights))


benchmark_backtest = Backtest(
    universe=universe,
    strategy=index_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

ewma_backtest = Backtest(
    universe=universe,
    strategy=ewma_strategy,
    market_view=market_view,
    initial_portfolio=pf,
)

In [3]:
print(
    all(
        x == 0
        for x in ewma_backtest.market_view.prices.close.by_security.as_df().row(0)
    )
)
ewma_results = ewma_backtest.run()

False


In [4]:
print(f"total return: {(ewma_results.total_return - 1) * 100:.2f}%")
sorted_weights = sorted(
    ewma_backtest._current_portfolio.holdings.items(), key=lambda x: x[1], reverse=True
)
sorted_weights

total return: 33.76%


[('ROST', 0.002298521361424705),
 ('ALGN', 0.002246915647448132),
 ('DHI', 0.002226566692873191),
 ('PCAR', 0.002209710947551594),
 ('COO', 0.002208659427228553),
 ('LEN', 0.002204290027134131),
 ('BLDR', 0.002203879877356062),
 ('MHK', 0.0021989919411689583),
 ('IR', 0.0021931823681087975),
 ('IDXX', 0.0021922550413766024),
 ('ODFL', 0.0021899419192014844),
 ('SWK', 0.002189129954921239),
 ('KMX', 0.002184734733501337),
 ('HPQ', 0.002182539907214447),
 ('CTSH', 0.0021819139465342717),
 ('MGM', 0.0021801122919574635),
 ('PHM', 0.0021795229937186054),
 ('RVTY', 0.0021792939825691915),
 ('TECH', 0.002174345551836111),
 ('IP', 0.002172237553590539),
 ('DOW', 0.0021707110676780213),
 ('WST', 0.002168822403927159),
 ('IEX', 0.002167158529973039),
 ('ZTS', 0.0021671581790619495),
 ('LYB', 0.0021662084628275824),
 ('BIIB', 0.002165668883815586),
 ('MAR', 0.0021647351985388777),
 ('EMN', 0.0021646549598907934),
 ('ZBRA', 0.002163876233569035),
 ('ACN', 0.0021605861004198625),
 ('LUV', 0.002159

In [5]:
results = benchmark_backtest.run()
print(f"total return: {(results.total_return - 1) * 100:.2f}%")

# what a strategy

total return: -33.57%


In [6]:
len(market_view.tradable.by_period)

1536

In [7]:
import altair as alt

final_allocations = (
    ewma_results.weights.by_period[-1:]
    .by_security.as_df(show_periods=False)
    .lazy()
    .unpivot()
    .filter(pl.col("value") > 0)
    .collect()
)
alt.Chart(final_allocations).mark_bar().encode(
    x="variable",
    y=alt.Y(
        "value",
        scale=alt.Scale(
            domain=[
                final_allocations["value"].min() * 0.99,
                final_allocations["value"].max() * 1.01,
            ],
            clamp=True,
        ),
    ),
)

alt.Chart(...)

In [8]:
final_allocations_series = ewma_results.weights.by_period[-1]
final_allocations = pl.DataFrame(
    data={
        "security": final_allocations_series.names,
        "value": final_allocations_series.as_series(),
    }
).filter(pl.col("value") > 0)

alt.Chart(final_allocations).mark_bar().encode(
    x="security",
    y=alt.Y(
        "value",
        scale=alt.Scale(
            domain=[
                final_allocations["value"].min() * 0.99,
                final_allocations["value"].max() * 1.01,
            ],
            clamp=True,
        ),
    ),
)

alt.Chart(...)

In [22]:
nvda_pnl = list(
    (results.asset_returns.by_security["NVDA"].as_series() + 1)
    .cum_prod()
    .rolling_mean(10, min_samples=10)
)
print(nvda_pnl)

nvda_pnl_values = [x for x in nvda_pnl if x is not None]

print(nvda_pnl_values)

dates = ewma_backtest.market_view.periods.array

alt.Chart(pl.DataFrame({"pnl": nvda_pnl, "date": dates})).mark_line().encode(
    x="date",
    y=alt.Y(
        "pnl",
        scale=alt.Scale(
            domain=[min(nvda_pnl_values) * 0.98, max(nvda_pnl_values) * 1.02],
            clamp=True,
        ),
    ),
)

[None, None, None, None, None, None, None, None, None, 1.0312788365585572, 1.0368746307122796, 1.0420510618721202, 1.0454441257226081, 1.049565782681514, 1.0537340346308965, 1.0563434348934995, 1.0513533917751559, 1.051645678991299, 1.0516329778771962, 1.050311334847502, 1.0448680348589021, 1.0410767926972146, 1.0407336687063047, 1.041009013024055, 1.0415978211610415, 1.0420680209296904, 1.051713466744047, 1.0601516361127574, 1.071588916645868, 1.082166286508532, 1.1047697531208822, 1.1257211619063203, 1.1466641032823568, 1.1737492856633263, 1.1968144846349238, 1.2148091435417376, 1.2191764846467201, 1.2167026437304365, 1.2146312229096423, 1.2069301221383406, 1.1985766672615934, 1.1929173271394125, 1.1799211868399326, 1.1671326111850437, 1.1521328216583404, 1.140259227153366, 1.128466121515243, 1.1280552259307626, 1.1190833108249447, 1.1037107532948363, 1.0913288314401375, 1.0574278692661074, 1.0368322775060927, 1.0022281325314395, 0.9766763915294557, 0.9511373597211638, 0.937264349693

alt.Chart(...)

In [20]:
print(results.market.prices.close.by_security["NVDA"][0])
print(results.market.prices.close.periods[0])
results.market.prices.close.by_security["NVDA"][-1]

5.901750087738037
2020-01-03T00:00:00.000000


178.8800048828125

In [ ]:
results.allocation_history